In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.datasets import fetch_california_housing

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

In [ ]:
RANDOM_STATE = 42

In [ ]:
data = fetch_california_housing(as_frame=True)

X = data.data
y = data.target

## Сравнение дерева и леса

In [ ]:
dt = DecisionTreeRegressor()

cross_val_score(dt, X, y, cv=3, scoring='r2').mean()

In [ ]:
rf = RandomForestRegressor()

cross_val_score(rf, X, y, cv=3, scoring='r2').mean()

## Подбор гиперпараметров леса

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

In [ ]:
params = {'n_estimators': np.arange(10, 200, 20),
          'max_depth': np.arange(2, 20, 3)}

gs = GridSearchCV(RandomForestRegressor(), params, cv=3, scoring='r2', verbose=2)

gs.fit(Xtrain, ytrain)

In [ ]:
gs.best_score_, gs.best_estimator_

In [ ]:
params = {'max_features': [None, 'sqrt', 'log2']}

gs = GridSearchCV(RandomForestRegressor(n_estimators = 190, max_depth = 17), params, cv=3, scoring='r2', verbose=2)

gs.fit(Xtrain, ytrain)

In [ ]:
gs.best_score_, gs.best_estimator_

Теперь проверим полученную модель на тестовых данных

In [ ]:
model = RandomForestRegressor(n_estimators = 190, max_depth = 17, max_features='log2')

model.fit(Xtrain, ytrain)

pred = model.predict(Xtest)

r2_score(ytest, pred)

## Поведение модели при увеличении числа деревьев

In [ ]:
r2_train = []
r2_test = []

for n in range(2, 50, 5):
    model = RandomForestRegressor(n_estimators = n, n_jobs=-1)
    model.fit(Xtrain, ytrain)

    pred_train = model.predict(Xtrain)
    pred_test = model.predict(Xtest)

    r2_train.append(r2_score(ytrain, pred_train))
    r2_test.append(r2_score(ytest, pred_test))

plt.plot(r2_train, color='blue')
plt.plot(r2_test, color='red')
plt.xlabel('iteration')
plt.ylabel('r2')